<a href="https://colab.research.google.com/github/DIPANJAN001/Dipanjanpatra/blob/master/StepFunc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np
import gym
from gym.utils import seeding
from gym.spaces import Space, Discrete, MultiDiscrete,  Box
from gym.spaces.space import Space
import numpy as np

### Environment

In [39]:
class MyEnv(gym.Env):
    
      def __init__(self):
        
        self.action_space = Discrete(4)
        self.observation_space=Box(low=np.array([0]),high=np.array([50]),dtype=np.float32) 
        self.battery_efficiency=0.3
        self.battery_cap=30
        self.max_battery_capacity=50
        #self.reward, self.done, self.info= None, None, None
        #self.grid_import=0
        #self.grid_export=0
        self.info=None
      def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

      def _step(self,action):
        grid_import_=0
        grid_export_=0
        done=False
        load=np.random.randint(60,110)
        power=np.random.randint(80,130)
        battery_state=self.battery_cap
        net_load=max((load-power),0)
        net_rem=max(net_load-(self.battery_efficiency*self.battery_cap),0)
        info={}
        reward=0
        battery_chng=0
        
        if action== 0:
          grid_export_=self.battery_cap+power-load-self.max_battery_capacity # grid export
          battery_chng=abs(self.battery_efficiency*self.max_battery_capacity)#change in battery
          self.battery_cap=min(max(self.battery_efficiency*self.max_battery_capacity,0),50) # battery charge
          info={
              'grid_export':grid_export_,
              'battery_change':battery_chng,
              'grid_import':0
          }
        if action== 1:
          battery_chng=abs(self.battery_efficiency*(power-load))#change in battery
          self.battery_cap=min(max(self.battery_cap + self.battery_efficiency*(power-load),0),50)# only battery charge
          info={
              'grid_export':0,
              'battery_change':battery_chng,
              'grid_import':0
          }
        if action== 2:
          grid_import_=net_rem #grid import
          battery_chng=abs(self.battery_efficiency*self.battery_cap)#change in battery
          self.battery_cap=min(max(self.battery_cap-self.battery_efficiency*self.battery_cap,0),50)# battery discharge
          info={
              'grid_export':0,
              'battery_change':battery_chng,
              'grid_import':grid_import_
             }
        if action== 3:
          battery_chng=abs(self.battery_efficiency*(load-power))#change in battery
          self.battery_cap=min(max(self.battery_cap-(load-power),0),50) #only battery discharge
          info={
              'grid_export':0,
              'battery_change':battery_chng,
              'grid_import':0
                    }     
        
        reward=grid_export_*8-grid_import_*10-(battery_chng*0.5)
        
        if(self.battery_cap<10.0):
              done=True
      
        return battery_state,load,power,reward, done,info
        
      def _reset(self):
        self.battery_cap=30
        #self.done=False
        return self.battery_cap


In [40]:
env=MyEnv()

### Agent

In [41]:
for e in range(5): #Episode
    #Play 20 episodes 
    score = 0
    observation = env._reset()
    #print(observation)
    done=False
    while not done:
        #env.render()
        action = env.action_space.sample()
        print(action)
        observation1,observation2,observation3,reward,done,info = env._step(action)
        print(observation1,observation2,observation3)
        #print(info)
        score+=round(reward)
        
        #print(info['load'],info['power'])
      
    print('episode:{} Profit:{}'.format(e,score))


3
30 78 122
1
50 99 94
1
48.5 88 87
0
48.2 62 119
1
15.0 83 117
1
25.2 60 84
2
32.4 100 109
2
22.68 70 86
3
15.876000000000001 82 99
3
32.876000000000005 71 101
0
50 105 113
2
15.0 102 92
1
10.5 94 119
0
18.0 81 119
1
15.0 70 108
1
26.4 90 107
2
31.5 70 94
0
22.05 89 96
3
15.0 89 120
1
46.0 94 91
2
45.1 105 82
3
31.57 77 124
2
50 108 89
0
35.0 96 114
3
15.0 108 82
episode:0 Profit:99
0
30 86 117
2
15.0 90 103
1
10.5 73 113
0
22.5 76 96
2
15.0 93 93
2
10.5 107 126
episode:1 Profit:0
2
30 90 108
0
21.0 75 100
3
15.0 104 119
0
30.0 104 119
0
15.0 70 127
0
15.0 87 108
2
15.0 108 118
3
10.5 76 119
1
50 109 125
2
50 88 95
1
35.0 80 115
1
45.5 91 87
1
44.3 92 101
1
47.0 109 106
2
46.1 103 108
3
32.27 94 122
0
50 93 110
0
15.0 105 103
3
15.0 85 92
1
22.0 95 86
3
19.3 100 104
2
23.3 108 112
0
16.310000000000002 68 122
1
15.0 62 117
2
31.5 82 129
3
22.05 96 107
3
33.05 77 114
1
50 89 108
2
50 108 82
1
35.0 73 126
2
50 65 84
0
35.0 108 99
0
15.0 86 122
2
15.0 103 111
1
10.5 88 119
1
19.7999999999

In [75]:
class MyEnv3(gym.Env):
    
      def __init__(self):
        
        self.action_space = Discrete(4)
        self.observation_space=Box(low=np.array([0]),high=np.array([50]),dtype=np.float32) 
        #self.battery_efficiency=0.3
        self.battery_cap=30
        self.max_battery_discharge=15
        self.max_battery_charge=10
        self.max_battery_capacity=50
        #self.reward, self.done, self.info= None, None, None
        #self.grid_import=0
        #self.grid_export=0
        self.info=None
      def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

      def _step(self,action):
        grid_import_=0
        grid_export_=0
        done=False
        load=np.random.randint(100,150)
        pv=np.random.randint(80,130)
        battery_state=self.battery_cap
        #net_load=max((load-power),0)
        #net_rem=max(net_load-(self.battery_efficiency*self.battery_cap),0)
        info={}
        reward=0
        #battery_chng=0
        
        if action== 0:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= min(self.battery_cap+min(self.max_battery_charge,pv-load),50)
          grid_import_=0
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 1:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= min(self.battery_cap+min(self.max_battery_charge,pv-load),50)
          grid_import_=0
          # only battery charge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 2:
            grid_export_=0
            self.battery_cap= min(max(self.battery_cap-self.max_battery_discharge,0),50)# battery discharge
            grid_import_=max(load-pv-self.max_battery_discharge,0)
            info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
             }
        if action== 3:
          grid_export_=0
          self.battery_cap= min(max(self.battery_cap-min(self.max_battery_discharge,load-pv),0),50)#change in battery
          grid_import_=max(load-pv-self.max_battery_discharge,0)
         #only battery discharge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
                    }     
        
        reward=grid_export_*8-grid_import_*10
        
        if(self.battery_cap<7.0):
              done=True
      
        return battery_state,self.battery_cap,load,pv,reward, done,info
        
      def _reset(self):
        self.battery_cap=30
        #self.done=False
        return self.battery_cap

In [76]:
env=MyEnv3()

In [77]:
for e in range(120): #Episode
    #Play 20 episodes 
    score = 0
    observation = env._reset()
    #print(observation)
    done=False
    while not done:
        #env.render()
        action = env.action_space.sample()
        print(action)
        observation1,observation2,observation3,observation4,reward,done,info = env._step(action)
        print(observation1,observation2,observation3,observation4)
        print(info)
        score+=round(reward)
        
        #print(info['load'],info['power'])
      
    print('episode:{} Profit:{}'.format(e,score))

1
30 17 133 120
{'grid_export': 0, 'grid_import': 0}
3
17 2 135 116
{'grid_export': 0, 'grid_import': 4}
episode:0 Profit:-40
3
30 15 147 108
{'grid_export': 0, 'grid_import': 24}
2
15 0 142 113
{'grid_export': 0, 'grid_import': 14}
episode:1 Profit:-380
3
30 15 116 100
{'grid_export': 0, 'grid_import': 1}
3
15 0 132 99
{'grid_export': 0, 'grid_import': 18}
episode:2 Profit:-190
2
30 15 134 89
{'grid_export': 0, 'grid_import': 30}
3
15 0 129 114
{'grid_export': 0, 'grid_import': 0}
episode:3 Profit:-300
2
30 15 104 91
{'grid_export': 0, 'grid_import': 0}
3
15 0 100 85
{'grid_export': 0, 'grid_import': 0}
episode:4 Profit:0
3
30 15 138 104
{'grid_export': 0, 'grid_import': 19}
0
15 16 128 129
{'grid_export': 0, 'grid_import': 0}
2
16 1 119 97
{'grid_export': 0, 'grid_import': 7}
episode:5 Profit:-260
0
30 -25 149 94
{'grid_export': 0, 'grid_import': 0}
episode:6 Profit:0
0
30 -21 138 87
{'grid_export': 0, 'grid_import': 0}
episode:7 Profit:0
3
30 15 123 81
{'grid_export': 0, 'grid_impor